# Reliability calculations with FORM

In this example, we will demonstrate how to perform reliability calculations using the First Order Reliability Method (FORM).

### Define model

First, let's import the necessary package:

In [1]:
from streams import *

Next, we define a simple limit state function: 

$Z = 1.9 - (a+b)$

This is a linear model involving two variables, $a$ and $b$.

In [2]:
def linear(a, b):
    return 1.9 - (a+b)

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [3]:
project = ReliabilityProject()
project.model = linear

We assume that variables $a$ and $b$ are uniformly distributed over the interval $[-1, 1]$. This is defined as follows:

In [4]:
project.variables["a"].distribution = DistributionType.uniform
project.variables["a"].minimum = -1
project.variables["a"].maximum = 1

project.variables["b"].distribution = DistributionType.uniform
project.variables["b"].minimum = -1
project.variables["b"].maximum = 1

### Define reliability method

Next, we define the reliability method: `form`. We choose the calculation settings: `relaxation_factor`, `maximum_iterations` and `variation_coefficient`. 

In [5]:
project.settings.reliability_method = ReliabilityMethod.form

project.settings.relaxation_factor = 0.75
project.settings.maximum_iterations = 50
project.settings.variation_coefficient = 0.02

### Perform calculations

We use `project.run()` to execute the reliability analysis:

In [6]:
project.run()

Exception ignored on calling ctypes callback function: <function ReliabilityProject._initialize at 0x000001F2F5D22430>
Traceback (most recent call last):
  File "C:\Users\bkm\Anaconda3\lib\site-packages\streams\interface.py", line 178, in Execute
    lib.Execute(ctypes.c_int(id_), bytes(method_, 'utf-8'))
TypeError: _initialize() missing 1 required positional argument: 'self'


The results are written to `project.design_point` and consist of:
* reliability index $\beta$
* failure probability $P_f$
* influence coefficients $\alpha$-values
* design point $x$-values
* information about the convergence of FORM


In [7]:
def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")
        
    print(f"Model runs = {dp.total_model_runs}")

read_results(project.design_point)

Beta = 2.772351852475011
Probability of failure = 0.0027826422680776335
a: alpha = -0.7071067811865476, x = 0.9500449635563615
b: alpha = -0.7071067811865476, x = 0.9500449635563615
Converged (convergence = 0.004050836299007645 < 0.02)
Model runs = 21


### Correlated variables

In the above example, variables $a$ and $b$ are independent. However, it is also possible to introduce a correlation between $a$ and $b$. The example below demonstrates this with a partial correlation coefficient of $0.8$.

In [8]:
project.correlation_matrix[("a", "b")] = 0.8

project.run()

read_results(project.design_point)


Exception ignored on calling ctypes callback function: <function ReliabilityProject._initialize at 0x000001F2F5D22430>
Traceback (most recent call last):
  File "C:\Users\bkm\Anaconda3\lib\site-packages\streams\interface.py", line 178, in Execute
    lib.Execute(ctypes.c_int(id_), bytes(method_, 'utf-8'))
TypeError: _initialize() missing 1 required positional argument: 'self'


Beta = 2.0665776344233597
Probability of failure = 0.01938698503961158
a: alpha = -0.9456009795757881, x = 0.9493174459367089
b: alpha = -0.3253287374722835, x = 0.950784099184725
Converged (convergence = 0.0038834889996542717 < 0.02)
Model runs = 21
